In [ ]:
import pandas as pd

df_origin = pd.DataFrame({
    'corp_id':    [111, 111, 111, 111, 112, 112, 112, 115],
    'corp_name':  ['humo', 'humo', 'humo', 'humo', 'zypl', 'zypl', 'zypl', 'freedom'],
    'client_id':  [201, 201, 202, 203, 204, 205, 206, 208],
    'name':       ['Азим', 'Азим', 'Фаррух', 'Абубакр', 'Джамшед', 'Нигина', 'Мадина', 'Рахим'],
    'roles':      ['Директор', 'Мухосиб', 'Мухосиб', 'Мухосиб', 'Директор', 'Мухосиб', 'Мухосиб', 'Директор'],
    'begin_date': ['2024-06-01', '2023-06-01', '2023-09-01', '2024-10-01', '2025-01-01', '2023-05-01', '2022-07-01', '2025-04-01'],
    'end_date':   [None, None, '2024-09-01', None, None, '2025-06-01', '2024-05-01', None]
})

df_origin['begin_date'] = pd.to_datetime(df_origin['begin_date'])
df_origin['end_date'] = pd.to_datetime(df_origin['end_date'])


df_cat = pd.DataFrame({
    'corp_id': [111, 111, 111, 112, 112, 112],
    'corp_name': ['humo', 'humo', 'humo', 'zypl', 'zypl', 'zypl'],
    'client_id': [201, 202, 203, 204, 205, 206],
    'name': ['Азим', 'Фаррух', 'Абубакр', 'Джамшед', 'Нигина', 'Мадина'],
    'roles': ['Директор', 'Мухосиб', 'Мухосиб', 'Директор', 'Мухосиб', 'Мухосиб'],
    'begin_date': ['2024-06-01', '2023-09-01', '2024-10-01', '2025-01-01', '2023-05-01', '2022-07-01'],
    'end_date': [None, '2024-09-01', None, None, '2025-06-01', '2024-05-01'],
    'is_active': [1, 0, 1, 1, 0, 0]
})

df_cat['begin_date'] = pd.to_datetime(df_cat['begin_date'])
df_cat['end_date'] = pd.to_datetime(df_cat['end_date'])



df_clean = df_origin.copy()

mask_invalid_dates = (df_clean['end_date'].notna()) & (df_clean['begin_date'] > df_clean['end_date'])
df_clean = df_clean[~mask_invalid_dates]



df_clean['is_active'] = df_clean['end_date'].isna().astype(int)


df_active = df_clean[df_clean['is_active'] == 1]

dups = (
    df_active.groupby(['corp_id', 'client_id'])
    .filter(lambda x: len(x) > 1)
)

df_duplic = (
    dups.sort_values('begin_date', ascending=False)
        .groupby(['corp_id', 'client_id'])
        .head(1)
        .reset_index(drop=True)
)



df_new = df_clean.merge(
    df_cat[['corp_id', 'client_id']],
    on=['corp_id', 'client_id'],
    how='left',
    indicator=True
)

df_new = df_new[df_new['_merge'] == 'left_only'].drop(columns=['_merge'])

df_cat_updated = pd.concat([df_cat, df_new], ignore_index=True)


if not df_duplic.empty:
    df_cat_updated = df_cat_updated.set_index(['corp_id', 'client_id'])
    df_duplic_idx = df_duplic.set_index(['corp_id', 'client_id'])

    df_cat_updated.update(df_duplic_idx)

    df_cat_updated = df_cat_updated.reset_index()


df_cat_updated.to_excel("persons_pos.xlsx", index=False)

df_cat_updated


,corp_id,client_id,corp_name,name,roles,begin_date,end_date,is_active
0,111,201,humo,Азим,Директор,2024-06-01,NaT,1
1,111,202,humo,Фаррух,Мухосиб,2023-09-01,2024-09-01,0
2,111,203,humo,Абубакр,Мухосиб,2024-10-01,NaT,1
3,112,204,zypl,Джамшед,Директор,2025-01-01,NaT,1
4,112,205,zypl,Нигина,Мухосиб,2023-05-01,2025-06-01,0
5,112,206,zypl,Мадина,Мухосиб,2022-07-01,2024-05-01,0
6,115,208,freedom,Рахим,Директор,2025-04-01,NaT,1
